In [1]:
import pandas as pd
import numpy as np

The sample used in this study contains firms on Vietnam stock exchange from 2002 to 2019. I exclude the following observations from the sample: **utility and financial firms** , **firms with non-positive total assets or sales**, **firms that are not traded on HNX, UPCOM, or HSX**, _firms with share codes other than 10 and 11_, firms with fewer than 100 daily stock price records during a fiscal year, and firms without sufficient data to calculate the control variables described below. In addition, I follow Love et al. (2007) and remove observations that imply trade credit of longer than 1 year. The final sample consists of 129,177 firm-year observations with 13,712 unique firms.

In [2]:
stockprices = pd.read_csv("StockPrices/CafeF.UPCOM.Upto02.10.2020.csv")\
                .append(pd.read_csv("StockPrices/CafeF.HNX.Upto02.10.2020.csv"))\
                .append(pd.read_csv("StockPrices/CafeF.HSX.Upto02.10.2020.csv"))

In [3]:
stockprices.columns=["TICKER","DATE","OPEN","HIGH","LOW","CLOSE","VOLUME"]
stockprices = stockprices[(stockprices.DATE < 20200101) & (stockprices.DATE > 20011231)]
stockprices.drop(columns=["OPEN","HIGH","LOW"],inplace=True)
stockprices.drop_duplicates(inplace=True)
stockprices.reset_index(inplace=True,drop=True)
stockprices.head()

,TICKER,DATE,CLOSE,VOLUME
0,ABC,20191231,9.8000,300
1,ABI,20191231,30.6968,10300
2,ABR,20191231,11.7000,1501
3,ACV,20191231,75.0000,81718
4,ADG,20191231,42.2776,18900


In [4]:
companies = pd.read_csv("Companies.csv")
companies.head()

/home/akizminet/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (0,1,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,TICKER,NAME,PRICE,EXCHANGE,URL
0,A32,Công ty cổ phần 32,39.00,Upcom,/upcom/A32-cong-ty-co-phan-32.chn
1,AAA,Công ty Cổ phần Nhựa An Phát Xanh,12.45,HSX,/hose/AAA-cong-ty-co-phan-nhua-an-phat-xanh.chn
2,AAM,Công ty Cổ phần Thủy sản Mekong,11.70,HSX,/hose/AAM-cong-ty-co-phan-thuy-san-mekong.chn
3,AAV,Công ty Cổ phần Việt Tiên Sơn Địa ốc,8.70,HNX,/hastc/AAV-cong-ty-co-phan-viet-tien-son-dia-o...
4,ABC,Công ty cổ phần Truyền thông VMG,13.90,Upcom,/upcom/ABC-cong-ty-co-phan-truyen-thong-vmg.chn


# Liquidity measures
**The illiquidity measure proposed by Amihud (2002) is used as the primary measure of stock liquidity in this study.** This measure is widely employed in the literature and has been demonstrated to be an appropriate proxy for stock illiquidity. For example, Goyenko et al. (2009) document that among 12 proxies that use daily data, the Amihud illiquidity measure most accurately captures price impact. Hasbrouck (2009) shows that, compared to other daily proxies, the Amihud illiquidity measure is the one most strongly correlated with a TAQ-based price impact coefficient. In addition, Fong et al. (2017) find that the Amihud illiquidity measure is the best daily cost-per-dollar-volume proxy. The Amihud illiquidity measure is calculated as the daily ratio of the absolute value of stock returns to dollar volume, averaged over firm i's fiscal year t:

$$\text{Amihud Illiquidity}_{i,t}=\dfrac{1}{D_{i,t}}\sum_{d=1}^{D}\dfrac{\left |\text{Ret}_{t,d}  \right |}{\text{Dollar Volume}_{i,d}}$$

Where $Ret$ and $\text{Dollar Volume}$ are the return and dollar volume of firm i on day d, respectively, and D is the total number of trading days during firm i's fiscal year t.

Since the distribution of the Amihud illiquidity measure is highly skewed, I follow Edmans et al.'s (2013) approach to modify the Amihud illiquidity measure by taking the natural logarithm of (Amihud illiquidity plus one). In addition, for the convenience of interpreting the empirical results, I multiply the modified Amihud illiquidity measure by −1 and name this measure “LiqAM”. Specifically, $LiqAM$ is defined as $–ln(\text{Amihud illiquidity} + 1)$. A higher value of $LiqAM$ is associated with a higher level of stock liquidity.


In [5]:
filtered_stocks = companies["TICKER"].to_frame().merge(stockprices, on='TICKER',how='inner')

In [6]:
filtered_stocks['YEAR'] = filtered_stocks['DATE']//10000
filtered_stocks.head()

,TICKER,DATE,CLOSE,VOLUME,YEAR
0,A32,20191224,27.2860,400,2019
1,A32,20191223,26.7994,200,2019
2,A32,20191220,26.7994,500,2019
3,A32,20191217,27.7236,100,2019
4,A32,20191126,25.8753,100,2019


In [7]:
lmstat = filtered_stocks[['TICKER','YEAR']]\
            .groupby(by=['TICKER','YEAR'])\
            .size().reset_index(name='counts')
lmstat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9306 entries, 0 to 9305
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TICKER  9306 non-null   object
 1   YEAR    9306 non-null   int64 
 2   counts  9306 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 218.2+ KB


In [8]:
# remove firms with fewer than 100 daily stock price records during a fiscal year
filtered_tickers = lmstat[lmstat.counts >= 100]

In [9]:
filtered_stocks= filtered_stocks.merge(filtered_tickers, on=['TICKER','YEAR'],how='inner')

In [10]:
filtered_stocks['VND_VOLUME'] = filtered_stocks['CLOSE']*filtered_stocks['VOLUME']*1000

In [11]:
filtered_stocks.sort_values(by=['TICKER','DATE'],inplace=True)

In [12]:
lstgr = []
for name,gr in filtered_stocks.groupby(['TICKER','YEAR']):
    print(name,end=';')
    gr['PREV_CLOSE'] = gr['CLOSE'].shift(1,fill_value=gr['CLOSE'].iloc[0])
    gr['RET'] = np.log(gr['CLOSE']/gr['PREV_CLOSE'])
    gr['RET_ON_VOL'] = gr['RET']/gr["VND_VOLUME"]
    lstgr.append(gr)
liquid_measure = pd.concat(lstgr)

('AAA', 2017);('AAA', 2018);('AAA', 2019);('AAM', 2010);('AAM', 2011);('AAM', 2012);('AAM', 2013);('AAM', 2014);('AAM', 2015);('AAM', 2016);('AAM', 2017);('AAM', 2018);('AAM', 2019);('AAV', 2018);('AAV', 2019);('ABC', 2017);('ABC', 2018);('ABC', 2019);('ABR', 2019);('ABT', 2009);('ABT', 2010);('ABT', 2011);('ABT', 2012);('ABT', 2013);('ABT', 2014);('ABT', 2015);('ABT', 2016);('ABT', 2017);('ABT', 2018);('ABT', 2019);('ACC', 2010);('ACC', 2011);('ACC', 2012);('ACC', 2013);('ACC', 2014);('ACC', 2015);('ACC', 2016);('ACC', 2017);('ACC', 2018);('ACC', 2019);('ACE', 2010);('ACE', 2016);('ACE', 2017);('ACE', 2018);('ACL', 2008);('ACL', 2009);('ACL', 2010);('ACL', 2011);('ACL', 2012);('ACL', 2013);('ACL', 2014);('ACL', 2015);('ACL', 2016);('ACL', 2017);('ACL', 2018);('ACL', 2019);('ACM', 2015);('ACM', 2016);('ACM', 2017);('ACM', 2018);('ACM', 2019);('ACV', 2017);('ACV', 2018);('ACV', 2019);('ADP', 2012);('ADP', 2016);('ADP', 2017);('ADP', 2018);('ADS', 2016);('ADS', 2017);('ADS', 2018);('ADS'

;('CJC', 2010);('CJC', 2011);('CJC', 2014);('CKA', 2019);('CKD', 2017);('CLC', 2007);('CLC', 2008);('CLC', 2009);('CLC', 2010);('CLC', 2011);('CLC', 2012);('CLC', 2013);('CLC', 2014);('CLC', 2015);('CLC', 2016);('CLC', 2017);('CLC', 2018);('CLC', 2019);('CLG', 2010);('CLG', 2011);('CLG', 2012);('CLG', 2013);('CLG', 2014);('CLG', 2015);('CLG', 2016);('CLG', 2017);('CLG', 2018);('CLG', 2019);('CLH', 2016);('CLH', 2017);('CLH', 2019);('CLL', 2014);('CLL', 2015);('CLL', 2016);('CLL', 2017);('CLL', 2018);('CLL', 2019);('CLS', 2009);('CLS', 2010);('CLS', 2011);('CLS', 2012);('CMC', 2007);('CMC', 2008);('CMC', 2009);('CMC', 2010);('CMC', 2011);('CMC', 2012);('CMC', 2013);('CMC', 2014);('CMG', 2010);('CMG', 2011);('CMG', 2012);('CMG', 2013);('CMG', 2014);('CMG', 2015);('CMG', 2016);('CMG', 2017);('CMG', 2018);('CMG', 2019);('CMN', 2018);('CMT', 2019);('CMV', 2010);('CMV', 2011);('CMV', 2012);('CMV', 2013);('CMV', 2014);('CMV', 2015);('CMV', 2016);('CMX', 2011);('CMX', 2012);('CMX', 2013);('CMX

;('DRL', 2015);('DRL', 2016);('DRL', 2017);('DRL', 2018);('DRL', 2019);('DSC', 2018);('DSC', 2019);('DSP', 2019);('DTA', 2010);('DTA', 2011);('DTA', 2012);('DTA', 2013);('DTA', 2014);('DTA', 2015);('DTA', 2016);('DTA', 2017);('DTA', 2018);('DTA', 2019);('DTC', 2007);('DTC', 2008);('DTC', 2009);('DTC', 2010);('DTC', 2011);('DTC', 2012);('DTC', 2014);('DTC', 2015);('DTC', 2018);('DTD', 2018);('DTD', 2019);('DTI', 2018);('DTL', 2010);('DTL', 2011);('DTL', 2012);('DTL', 2013);('DTL', 2014);('DTL', 2015);('DTL', 2016);('DTL', 2017);('DTL', 2018);('DTL', 2019);('DTT', 2007);('DTT', 2008);('DTT', 2009);('DTT', 2010);('DTT', 2011);('DTT', 2012);('DTT', 2013);('DTT', 2014);('DTT', 2016);('DTT', 2017);('DTT', 2018);('DVN', 2017);('DVN', 2018);('DVN', 2019);('DVP', 2010);('DVP', 2011);('DVP', 2012);('DVP', 2013);('DVP', 2014);('DVP', 2015);('DVP', 2016);('DVP', 2017);('DVP', 2018);('DVP', 2019);('DXG', 2010);('DXG', 2011);('DXG', 2012);('DXG', 2013);('DXG', 2014);('DXG', 2015);('DXG', 2016);('DXG

;('HMC', 2015);('HMC', 2016);('HMC', 2017);('HMC', 2018);('HMC', 2019);('HMH', 2010);('HMH', 2011);('HMH', 2012);('HMH', 2013);('HMH', 2014);('HMH', 2015);('HMH', 2016);('HMH', 2017);('HMH', 2018);('HMH', 2019);('HMS', 2017);('HMS', 2018);('HMS', 2019);('HNA', 2018);('HNA', 2019);('HND', 2017);('HND', 2018);('HND', 2019);('HNF', 2016);('HNF', 2017);('HNG', 2015);('HNG', 2016);('HNG', 2017);('HNG', 2018);('HNG', 2019);('HNI', 2018);('HNM', 2007);('HNM', 2008);('HNM', 2009);('HNM', 2010);('HNM', 2011);('HNM', 2012);('HNM', 2013);('HNM', 2014);('HNM', 2015);('HNM', 2016);('HNM', 2017);('HNM', 2018);('HNP', 2017);('HOM', 2009);('HOM', 2010);('HOM', 2011);('HOM', 2012);('HOM', 2013);('HOM', 2014);('HOM', 2015);('HOM', 2016);('HOM', 2017);('HOM', 2018);('HOM', 2019);('HPB', 2010);('HPB', 2011);('HPB', 2012);('HPB', 2013);('HPB', 2014);('HPD', 2016);('HPG', 2008);('HPG', 2009);('HPG', 2010);('HPG', 2011);('HPG', 2012);('HPG', 2013);('HPG', 2014);('HPG', 2015);('HPG', 2016);('HPG', 2017);('HPG

;('LAF', 2004);('LAF', 2005);('LAF', 2006);('LAF', 2007);('LAF', 2008);('LAF', 2009);('LAF', 2010);('LAF', 2011);('LAF', 2012);('LAF', 2013);('LAF', 2014);('LAF', 2015);('LAF', 2016);('LAF', 2017);('LAF', 2018);('LAF', 2019);('LAS', 2012);('LAS', 2013);('LAS', 2014);('LAS', 2015);('LAS', 2016);('LAS', 2017);('LAS', 2018);('LAS', 2019);('LBM', 2007);('LBM', 2008);('LBM', 2009);('LBM', 2010);('LBM', 2011);('LBM', 2012);('LBM', 2013);('LBM', 2014);('LBM', 2015);('LBM', 2016);('LBM', 2017);('LBM', 2018);('LBM', 2019);('LCC', 2010);('LCC', 2011);('LCC', 2012);('LCG', 2009);('LCG', 2010);('LCG', 2011);('LCG', 2012);('LCG', 2013);('LCG', 2014);('LCG', 2015);('LCG', 2016);('LCG', 2017);('LCG', 2018);('LCG', 2019);('LCM', 2012);('LCM', 2013);('LCM', 2014);('LCM', 2015);('LCM', 2016);('LCM', 2017);('LCM', 2018);('LCM', 2019);('LCS', 2010);('LCS', 2011);('LCS', 2012);('LCS', 2013);('LCS', 2014);('LCS', 2017);('LCS', 2018);('LDG', 2015);('LDG', 2016);('LDG', 2017);('LDG', 2018);('LDG', 2019);('LDP

;('NTP', 2008);('NTP', 2009);('NTP', 2010);('NTP', 2011);('NTP', 2012);('NTP', 2013);('NTP', 2014);('NTP', 2015);('NTP', 2016);('NTP', 2017);('NTP', 2018);('NTP', 2019);('NVL', 2017);('NVL', 2018);('NVL', 2019);('NVT', 2010);('NVT', 2011);('NVT', 2012);('NVT', 2013);('NVT', 2014);('NVT', 2015);('NVT', 2016);('NVT', 2017);('NVT', 2018);('NVT', 2019);('OCH', 2011);('OCH', 2012);('OCH', 2013);('OCH', 2014);('OCH', 2016);('OCH', 2017);('OCH', 2018);('OGC', 2010);('OGC', 2011);('OGC', 2012);('OGC', 2013);('OGC', 2014);('OGC', 2015);('OGC', 2016);('OGC', 2017);('OGC', 2018);('OGC', 2019);('OIL', 2018);('OIL', 2019);('ONE', 2008);('ONE', 2009);('ONE', 2010);('ONE', 2011);('ONE', 2012);('ONE', 2013);('ONE', 2014);('ONE', 2015);('ONE', 2016);('ONE', 2017);('ONE', 2018);('ONE', 2019);('OPC', 2009);('OPC', 2010);('OPC', 2011);('OPC', 2012);('OPC', 2013);('OPC', 2014);('OPC', 2015);('OPC', 2016);('OPC', 2017);('OPC', 2018);('OPC', 2019);('PAC', 2007);('PAC', 2008);('PAC', 2009);('PAC', 2010);('PAC

;('PXS', 2014);('PXS', 2015);('PXS', 2016);('PXS', 2017);('PXS', 2018);('PXS', 2019);('PXT', 2010);('PXT', 2011);('PXT', 2012);('PXT', 2013);('PXT', 2014);('PXT', 2015);('PXT', 2016);('PXT', 2017);('PXT', 2018);('PXT', 2019);('QBS', 2015);('QBS', 2016);('QBS', 2017);('QBS', 2018);('QBS', 2019);('QCC', 2011);('QCC', 2012);('QCC', 2014);('QCC', 2015);('QCC', 2016);('QCG', 2010);('QCG', 2011);('QCG', 2012);('QCG', 2013);('QCG', 2014);('QCG', 2015);('QCG', 2016);('QCG', 2017);('QCG', 2018);('QCG', 2019);('QHD', 2010);('QHD', 2011);('QHD', 2012);('QHD', 2014);('QHD', 2015);('QHD', 2016);('QHD', 2017);('QHD', 2018);('QHW', 2018);('QNC', 2008);('QNC', 2009);('QNC', 2010);('QNC', 2011);('QNC', 2012);('QNC', 2013);('QNC', 2014);('QNC', 2015);('QNC', 2016);('QNC', 2017);('QNC', 2018);('QNS', 2017);('QNS', 2018);('QNS', 2019);('QTC', 2009);('QTC', 2010);('QTC', 2011);('QTC', 2012);('QTC', 2013);('QTC', 2014);('QTC', 2015);('QTC', 2016);('QTC', 2017);('QTP', 2017);('QTP', 2018);('QTP', 2019);('RAL

;('SJC', 2014);('SJC', 2015);('SJC', 2016);('SJC', 2017);('SJC', 2018);('SJC', 2019);('SJD', 2007);('SJD', 2008);('SJD', 2009);('SJD', 2010);('SJD', 2011);('SJD', 2012);('SJD', 2013);('SJD', 2014);('SJD', 2015);('SJD', 2016);('SJD', 2017);('SJD', 2018);('SJD', 2019);('SJE', 2007);('SJE', 2008);('SJE', 2009);('SJE', 2010);('SJE', 2011);('SJE', 2012);('SJE', 2013);('SJE', 2014);('SJE', 2015);('SJE', 2016);('SJE', 2017);('SJE', 2018);('SJE', 2019);('SJF', 2017);('SJF', 2018);('SJF', 2019);('SJM', 2008);('SJM', 2009);('SJM', 2010);('SJM', 2011);('SJM', 2012);('SJM', 2013);('SJM', 2014);('SJM', 2015);('SJM', 2017);('SJM', 2018);('SJM', 2019);('SJS', 2006);('SJS', 2007);('SJS', 2008);('SJS', 2009);('SJS', 2010);('SJS', 2011);('SJS', 2012);('SJS', 2013);('SJS', 2014);('SJS', 2015);('SJS', 2016);('SJS', 2017);('SJS', 2018);('SJS', 2019);('SKG', 2014);('SKG', 2015);('SKG', 2016);('SKG', 2017);('SKG', 2018);('SKG', 2019);('SKH', 2018);('SKH', 2019);('SKV', 2018);('SKV', 2019);('SLS', 2013);('SLS

;('TMP', 2019);('TMS', 2002);('TMS', 2003);('TMS', 2004);('TMS', 2005);('TMS', 2006);('TMS', 2007);('TMS', 2008);('TMS', 2009);('TMS', 2010);('TMS', 2011);('TMS', 2012);('TMS', 2013);('TMS', 2014);('TMS', 2015);('TMS', 2016);('TMS', 2017);('TMS', 2018);('TMS', 2019);('TMT', 2010);('TMT', 2011);('TMT', 2012);('TMT', 2013);('TMT', 2014);('TMT', 2015);('TMT', 2016);('TMT', 2017);('TMT', 2018);('TMT', 2019);('TN1', 2019);('TNB', 2010);('TNB', 2011);('TNB', 2012);('TNB', 2013);('TNB', 2014);('TNB', 2015);('TNB', 2016);('TNB', 2017);('TNB', 2018);('TNB', 2019);('TNC', 2008);('TNC', 2009);('TNC', 2010);('TNC', 2011);('TNC', 2012);('TNC', 2013);('TNC', 2014);('TNC', 2015);('TNC', 2016);('TNC', 2017);('TNC', 2018);('TNC', 2019);('TND', 2017);('TND', 2018);('TND', 2019);('TNG', 2008);('TNG', 2009);('TNG', 2010);('TNG', 2011);('TNG', 2012);('TNG', 2013);('TNG', 2014);('TNG', 2015);('TNG', 2016);('TNG', 2017);('TNG', 2018);('TNG', 2019);('TNI', 2017);('TNI', 2018);('TNI', 2019);('TNM', 2010);('TNM

;('VFC', 2007);('VFC', 2008);('VFC', 2009);('VFC', 2010);('VFC', 2011);('VFC', 2012);('VFC', 2014);('VFC', 2015);('VFG', 2010);('VFG', 2011);('VFG', 2012);('VFG', 2013);('VFG', 2014);('VFG', 2015);('VFG', 2016);('VFG', 2017);('VFG', 2018);('VFG', 2019);('VFR', 2007);('VFR', 2008);('VFR', 2009);('VFR', 2010);('VFR', 2011);('VFR', 2012);('VFR', 2013);('VFR', 2014);('VFR', 2015);('VFR', 2016);('VGC', 2016);('VGC', 2017);('VGC', 2018);('VGC', 2019);('VGG', 2016);('VGG', 2017);('VGG', 2018);('VGG', 2019);('VGI', 2019);('VGP', 2007);('VGP', 2008);('VGP', 2009);('VGP', 2010);('VGP', 2011);('VGP', 2012);('VGP', 2013);('VGP', 2014);('VGP', 2015);('VGP', 2016);('VGP', 2018);('VGS', 2009);('VGS', 2010);('VGS', 2011);('VGS', 2012);('VGS', 2013);('VGS', 2014);('VGS', 2015);('VGS', 2016);('VGS', 2017);('VGS', 2018);('VGS', 2019);('VGT', 2017);('VGT', 2018);('VGT', 2019);('VGV', 2018);('VHC', 2008);('VHC', 2009);('VHC', 2010);('VHC', 2011);('VHC', 2012);('VHC', 2013);('VHC', 2014);('VHC', 2015);('VHC

In [13]:
liquid_measure.head()

,TICKER,DATE,CLOSE,VOLUME,YEAR,counts,VND_VOLUME,PREV_CLOSE,RET,RET_ON_VOL
747,AAA,20170103,16.0114,694080,2017,250,1.111319e+10,16.0114,0.000000,0.000000e+00
746,AAA,20170104,16.2067,740770,2017,250,1.200544e+10,16.0114,0.012124,1.009856e-12
745,AAA,20170105,16.5321,1057020,2017,250,1.747476e+10,16.2067,0.019879,1.137595e-12
744,AAA,20170106,15.9464,823010,2017,250,1.312405e+10,16.5321,-0.036071,-2.748455e-12
743,AAA,20170109,16.9227,777440,2017,250,1.315638e+10,15.9464,0.059423,4.516653e-12


In [14]:
Amihud_Illiquidity = liquid_measure[['TICKER','YEAR','RET_ON_VOL']].groupby(['TICKER','YEAR']).agg(['sum','count'])

In [16]:
Amihud_Illiquidity.head()

RET_ON_VOL      
                      sum count
TICKER YEAR                    
AAA    2017  1.055925e-11   250
       2018 -5.831967e-11   248
       2019 -1.047013e-11   250
AAM    2010 -2.251712e-09   250
       2011  4.322852e-07   246